In [45]:
class Globals:
    auth_token = '<YOUR_AUTH_TOKEN_HERE>'
    api_url = '<YOUR_API_URL>'
    contextMode = 'single_doc'
    docNames = ''
    domainName = ''
    summary = ''
    graph_suggestions = []
    code_blocks = []
    org = ""
    directory = ""
    filePath = ""
    files = {}
    folderName = ""

import json
import requests
import time
from datetime import datetime

In [46]:
def takeInputs():
    Globals.org = "org"
    Globals.directory = "directory"
    Globals.filePath = input("please enter the file path(full path) : ")
    Globals.files = {
        'file': open(Globals.filePath, 'rb')
    }

In [47]:
def upload():
    headers = {
        'Authorization': Globals.auth_token
    }


    data = {
        'org': Globals.org,
        'directory': Globals.directory,
        'contextMode': Globals.contextMode
    }
    
    print("Now uploading the file")

    response = requests.post(Globals.api_url+'upload', headers=headers, files=Globals.files, data=data, verify=False)

    if response.status_code == 200:
        Globals.docNames = response.json()['docNames']
        print('Success:', Globals.docNames)
    else:
        print('Failed:', response.status_code, response.text)

In [48]:
def generateDomainName():
    headers = {
        'Authorization': Globals.auth_token
    }

    inputs = f"Give me the domain of this dataset ${Globals.docNames} in one word. no additional information required."

    data = {
        'org': Globals.org,
        'contextMode': Globals.contextMode,
        'inputs': inputs
    }

    
    print("Now getting the domain name")
    
    response = requests.post(Globals.api_url+'stream', headers=headers, data=data)

    if response.status_code == 200:
        last_json = None
        for line in response.iter_lines():
            if line:
                line_str = line.decode('utf-8').strip()
                if line_str.startswith("data:"):
                    line_str = line_str[len("data:"):].strip()
                try:
                    last_json = json.loads(line_str)
                except json.JSONDecodeError:
                    print(f"Could not decode line to JSON: {line.decode('utf-8')}")
        if last_json is not None:
            Globals.domainName = last_json['generated_text'].strip().split("Source Document:")[0].strip()
            Globals.folderName = 'html_pages_' + Globals.domainName + '_' + str(datetime.now().timestamp()).split(".")[0]
            print("Domain name is " + Globals.domainName)
    else:
        print('Failed:', response.status_code, response.text)

In [49]:
def generateSummary():
    headers = {
        'Authorization': Globals.auth_token
    }

    inputs = f"Can you give me the summary of the dataset {Globals.docNames}"

    print("Now getting the summary")
    
    data = {
        'org': Globals.org,
        'contextMode': Globals.contextMode,
        'inputs': inputs
    }

    response = requests.post(Globals.api_url+'stream', headers=headers, data=data)

    if response.status_code == 200:
        last_json = None
        for line in response.iter_lines():
            if line:
                line_str = line.decode('utf-8').strip()
                if line_str.startswith("data:"):
                    line_str = line_str[len("data:"):].strip()
                try:
                    last_json = json.loads(line_str)
                except json.JSONDecodeError:
                    print(f"Could not decode line to JSON: {line.decode('utf-8')}")
        if last_json is not None:
            Globals.summary = last_json['generated_text'].strip().split("Source Document:")[0].strip()
            print(Globals.summary)
    else:
        print('Failed:', response.status_code, response.text)


In [50]:
def generateGraphs():
    headers = {
        'Authorization': Globals.auth_token
    }

    server_response_graphs = None

    inputs = f"""From the dataset based on the column names tell me 8 different graphs in points form in a dashboard {Globals.docNames} in a python variable with a sample code 
    (array of graphs in format graphs = ['graph name 1','graph name 2']) 
    ```python
    graphs = ['','']
    ```
    """

    data = {
        'org': Globals.org,
        'contextMode': Globals.contextMode,
        'inputs': inputs
    }
    
    print("Now getting the graph suggestions")

    response = requests.post(Globals.api_url+'stream', headers=headers, data=data)
    if response.status_code == 200:
        last_json = None
        for line in response.iter_lines():
            if line:
                line_str = line.decode('utf-8').strip()
                if line_str.startswith("data:"):
                    line_str = line_str[len("data:"):].strip()
                try:
                    last_json = json.loads(line_str)
                except json.JSONDecodeError:
                    print(f"Could not decode line to JSON: {line.decode('utf-8')}")
        if last_json is not None:
            server_response_graphs = last_json['generated_text'].strip()
#             print(last_json['generated_text'].strip())
            
            start_marker = "```python"
            end_marker = "``"
            code_block = ""
            if start_marker in server_response_graphs and end_marker in server_response_graphs:
                code_block = server_response_graphs.split(start_marker)[1].split(end_marker)[0].strip()
                print(code_block)

            Globals.graph_suggestions = [line.strip().strip("'") for line in code_block.splitlines() if "'" in line]

#             Globals.graph_suggestions = [suggestion.strip() for suggestion in server_response_graphs.split('\n') if suggestion.strip()]
        
            for idx, suggestion in enumerate(Globals.graph_suggestions, start=1):
                print(f"{idx}. {suggestion}")
    else:
        print('Failed:', response.status_code, response.text)
    print("Extracted Graph Suggestions:")
    print(Globals.graph_suggestions)

In [55]:
def generateCodes1():
    headers = {
        'Authorization': Globals.auth_token
    }
    
    Globals.code_blocks = []

    print("Now getting the graph codes")

    for graph in Globals.graph_suggestions:
        inputs =  f"""first read the {Globals.docNames} file and the file path is {Globals.filePath} 
        and then rename all the column names with no spaces in it and then based on these suggestions 
        give me python code using plotly dash (an example has been provided for your reference)
        only that generates an html page with the generated graph {graph} and print the html content and 
        save it in a file name {graph}.html that displays the graph for the suggestion {graph} and 
        save the graph in a html page with the name {graph} under new folder name 
        {Globals.folderName} (create a new folder if not present) and there is no need to run the dash application
        based on the given below example for the dataset {Globals.docNames} and 
        you are not matching the column name with the exact csv file
        
        
        sample code for saving the graph to a html page
        
        ```python
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import random


df = pd.DataFrame(data)

# Create a Plotly scatter plot
fig = px.scatter(df, x='x', y='y', title='Random Scatter Plot')

# Initialize the Dash app
app = dash.Dash(__name__)

# Layout of the web app
app.layout = html.Div([
    html.H1('Interactive Scatter Plot'),
    dcc.Graph(
        id='scatter-plot',
        figure=fig
    )
])

# Save the plot as an HTML file
fig.write_html('scatter_plot.html')


```

        """
        data = {
        'org': Globals.org,
        'contextMode': Globals.contextMode,
        'inputs': inputs
        }
        response = requests.post(Globals.api_url+'stream', headers=headers, data=data)

        if response.status_code == 200:
            last_json = None
            for line in response.iter_lines():
                if line:
                    line_str = line.decode('utf-8').strip()
                    if line_str.startswith("data:"):
                        line_str = line_str[len("data:"):].strip()
                    try:
                        last_json = json.loads(line_str)
                    except json.JSONDecodeError:
                        print(f"Could not decode line to JSON: {line.decode('utf-8')}")
            if last_json is not None:
                server_response_graphs = last_json['generated_text'].strip()
                print(server_response_graphs)
                start_marker = "```python"
                end_marker = "``"
        
                if start_marker in server_response_graphs and end_marker in server_response_graphs:
                    code_block = server_response_graphs.split(start_marker)[1].split(end_marker)[0].strip()
                    print(code_block)
                    Globals.code_blocks.append(code_block)
                else:
                    print("Code block markers not found in the response.")
        else:
            print('Failed:', response.status_code, response.text)

In [52]:
def executeCodes():
    print("Now running the code")
    for i in Globals.code_blocks:
        try:
#             print(i)
            exec(i)
        except Exception as e:
            print(e)

In [53]:
import os
from pathlib import Path

def generateHTMLPage(folder_path, output_file):
    # List all HTML files in the specified folder
    html_files = [f for f in os.listdir(folder_path) if f.endswith('.html')]
    
    # Initialize the HTML content
    combined_html = f"""
    <html>
    <head>
        <title>Combined HTML Files</title>
    </head>
    <body>
        <h1>Domain Name - {Globals.domainName}</h1>
        <p><strong>Summary:</strong> {Globals.summary}</p>
        <hr>
    """
    
    # For each HTML file, we will load its content and display it with the image name
    for html_file in html_files:
        # Load the HTML file content
        with open(os.path.join(folder_path, html_file), 'r', encoding='utf-8') as file:
            file_content = file.read()
        
        # Extract the image name from the HTML file's name (or adjust as per your requirements)
        image_name = Path(html_file).stem  # Use the stem (name without extension) as image name
        
        # Add the content of the HTML file and the image name to the combined HTML
        combined_html += f"""
        <div style="margin-bottom: 40px;">
            <p><strong>Image Name:</strong> {image_name}</p>
            <div>{file_content}</div>
        </div>
        <hr>
        """
    
    # Close the HTML document
    combined_html += """
    </body>
    </html>
    """
    
    # Save the combined HTML to the specified output file
    with open(output_file, 'w', encoding='utf-8') as output:
        output.write(combined_html)
    
    print(f"Combined HTML file saved to {output_file}")


In [54]:
takeInputs()
upload()
time.sleep(2)
generateDomainName()
time.sleep(2)
generateSummary()
time.sleep(2)
generateGraphs()
time.sleep(2)
generateCodes1()
time.sleep(2)
executeCodes()
time.sleep(2)
outputFileName = 'final_output_' + Globals.domainName + "_" + str(datetime.now().timestamp()).split(".")[0] + '.html'
generateHTMLPage(Globals.folderName , outputFileName)

please enter the file path(full path) : C:\Users\nredd\Downloads\digital_wallet_transactions.csv
Now uploading the file


C:\Users\nredd\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'hd7hkfzwuj16rz-8888.proxy.runpod.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings



Success: digital_wallet_transactions.csv
Now getting the domain name
Domain name is Finance
Now getting the summary
Based on the provided context, here is a summary of the dataset:

**Document Name:** org__digital_wallet_transactions

**Number of Pages:** 3 (but this seems to be a repetition, likely an error)

**Columns:** 15

**Columns Description:**

1. idx
2. transaction_id
3. user_id
4. transaction_date
5. product_category
6. product_name
7. merchant_name
8. product_amount
9. transaction_fee
10. cashback
11. loyalty_points
12. payment_method
13. transaction_status
14. merchant_id
15. device_type
16. location

**Note:** There seems to be an error in the provided context as the column names are repeated three times. The correct number of columns is 16, not 15.
Now getting the graph suggestions
graphs = [
    'Transaction Fee Distribution Over Time',
    'Product Category-wise Transactions',
    'Merchant-wise Transaction Amount',
    'Device Type-wise Transactions',
    'Location-wis

Based on the provided context, I will write the Python code using Plotly Dash to generate an HTML page with a graph displaying Device Type-wise Transactions.

```python
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

# Read the CSV file
df = pd.read_csv('C:\\Users\\nredd\\Downloads\\digital_wallet_transactions.csv')

# Rename columns with no spaces
df.columns = ['idx', 'transaction_id', 'user_id', 'transaction_date', 
              'product_category', 'product_name', 'erchant_name', 
              'product_amount', 'transaction_fee', 'cashback', 
              'loyalty_points', 'payment_method', 'transaction_status', 
              'erchant_id', 'device_type', 'location']

# Rename columns with no spaces in it
df.columns = df.columns.str.replace(' ', '_')

# Group by device type and calculate total transactions
fig = px.bar(df, x='device_type', y='idx', title='Device Type-wise Transactions')

# Save the

Here is the Python code that reads the `digital_wallet_transactions.csv` file, renames the column names with no spaces, and generates an HTML page with the graph "Payment Method-wise Cashback".

```python
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import os

# Read the CSV file
df = pd.read_csv('C:\\Users\\nredd\\Downloads\\digital_wallet_transactions.csv')

# Rename column names with no spaces
df.columns = [col.strip().replace(' ', '_') for col in df.columns if ' in col]

# Create a new column for transaction amount
df['transaction_amount'] = df['product_amount'] - df['transaction_fee']

# Group the data by payment method and calculate the total cashback
df_grouped = df.groupby('payment_method').agg({'cashback': 'um'})

# Plot a bar chart
fig = px.bar(df_grouped, x='payment_method', y='cashback', title='Payment Method-wise Cashback')

# Save the graph as an HTML file
folder_path = 'html_pages_Finance_1731478408'
if not os.path.exists(folder_

[Errno 2] No such file or directory: 'C:\\Users\\nredd\\html_pages_Finance_1731478408\\Device_Type-wise_Transactions.html'
Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['index', 'transactionId', 'userId', 'transactionDate', 'productCategory', 'productName', 'merchant_name', 'productAmount', 'transactionFee', 'cashbackEarned', 'loyaltyPoints', 'paymentMethod', 'transactionStatus', 'merchant_id', 'deviceType', 'location'] but received: transactionAmount
'Div' object has no attribute 'prettify'
unterminated string literal (detected at line 11) (<string>, line 11)
'charmap' codec can't decode byte 0x8f in position 313778: character maps to <undefined>
Combined HTML file saved to final_output_Finance_1731478502.html
